# Simpler integer matrices

One of the annoying things when teaching is that 
the **smartass** that sets the questions will just
take random integer coefficients without thinking
so you get a matrix that isn't nice to work with.


This notebook attempts to solve the problem of the poor instructor/grader.

- 3x3 rank 2 matrix with kernel and equation $ax + by + cz = 0$ for the image
- 3x3 invertible matrix with given determinant and hints on how to find its inverse
- a function for rendering an np.array as a LaTex pmatrix

---

## what you get?

def rand_rank2(max_int=6):<br>

    ''' returns 
        A rank 2,  3x3 matrix
        K vector basis for the kernel
        V vector orthogonal to the image
    '''

def rand_inv(high=3,
             D=[1,1,1])->np.array:<br>
    
    ''' returns an invertible matrix with
        - integer coefficients in the range [-high, high] 
        - determinant is the product of the elements of D
        - a list of row operations to put it in upper triangular form
        
        see https://en.wikipedia.org/wiki/Bruhat_decomposition
        
        
        
def rand_diagonalisable(max_int=3,
             D=[1,2,3])->(np.array,np.array):
    
    '''returns 
    
    3x3 matrix M with
    - integer coefficients in [-max_int,max_int]
    - eigenvalues from the list D
    
    3x3 matrix A with
    - integer coefficients
    - M = A^{-1}DA
    
    '''
        
    

---

Copyright 2022  Greg McShane (macbuse.github)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [1]:
import numpy as np

def m2t(X):
    def l2t(R):
        return  ' & '.join([str(_) for _ in R])
    
    body = ' \\\\ \n'.join([l2t(_) for _ in X ])
    print(f"\\begin{{pmatrix}}\n{body}\n\\end{{pmatrix}}")

In [3]:
def rand_rank2(max_int=6):
    '''returns 
        A rank 2,  3x3 matrix
        K vector basis for the kernel
        V vector orthogonal to the image
    '''
    
    def kill_factors(v):
        
        def gcd(b,a):
            while b:
                a,b = b,a%b
            return a

        x = sorted(list(abs(v)))
        a = x[2]
        for b in x[:2]:
            a = gcd(a,b)
            if a == 1 : return v
        if a == 0 : return v
        return v//a
    
    for _ in range(10):
        A = np.random.randint(-max_int, high=max_int,size=9).reshape(-1,3)
        
        # make sure the first 2 rows are LI for the next step below
        if np.all(np.cross(A[0],A[1])) == 0:continue
        
        # I don't want a column with all zeros
        if not np.any(np.all(A[:2,:] == 0,axis=0)) :
            
            # and I don't like common factors either
            A[0] = kill_factors(A[0])
            A[1] = kill_factors(A[1])
            break
    
    # the 3rd row is a combination of the other two
    V = np.random.choice([-5,-3,-2,-1,1,2,3,5], 3)
    V[2] = -1
    A[2] = V[:2] @ A[:2] 
    
    #kernel is easy to calculate
    K = kill_factors(np.cross(A[0],A[1]))
    
    #now shuffle the rows
    perm = [0,1,2]
    np.random.shuffle(perm)
    
    return A[perm], K, V[perm]

#f"{a}x + {b}y + {c}z = 0"

A,K,I = rand_rank2()
A,K, I

(array([[-22,   6,  -9],
        [  4,   0,   3],
        [  1,  -3,  -3]]),
 array([-3, -5,  4]),
 array([-1, -5, -2]))

In [160]:
import re

def tidy(ss):
    '''clean up a list of row operations'''
    ss = re.sub('\+ \-', '- ',ss)
    ss = re.sub('([\s|-])1L', lambda x: x[1] + 'L',ss)
    return ss

def ut(L:list, 
       D=[1,1,1])->np.array:
    
    x,y,z = L
    A  = np.diag(D).astype(int)
    A[0,1:] = x,y
    A[1,-1] = z
    return A

def rand_inv(max_int=3,
             D=[1,1,1])->np.array:
    
    ''' returns an invertible 3x3 matrix with
        - integer coefficients in the range [-high, high] 
        - determinant is the product of the elements of D
        - a list of row operations to put it in upper triangular form
        
        see https://en.wikipedia.org/wiki/Bruhat_decomposition
        
    '''
    
    X,Y = np.random.randint(-max_int,high=max_int,size=6).reshape(2,-1)
    coeffs = ut(X).T[[1,2,2],[0,0,1]]
    coeffs *= -1
    
    row_ops = ['L_2 <- L_2 + {}L_1',
           'L_3 <- L_3 + {}L_1', 
           'L_3 <- L_3 + {}L_2']
    row_ops = [ss.format(coeff) for coeff,ss in zip(coeffs, row_ops) if coeff != 0]
    
    return ut (X).T @ ut(Y, D=D), '\n'.join(row_ops)

A, B = rand_inv(D=[2,2,1],
                max_int=8)
print(A, '\n'*2 + tidy(B))

[[  2   7  -2]
 [ 10  37 -14]
 [-14 -41  -1]] 

L_2 <- L_2 - 5L_1
L_3 <- L_3 + 7L_1
L_3 <- L_3 - 4L_2


In [35]:
ss = 'L_3 <- L_3 + -1L_2'
re.sub('([\s|-])1L', lambda x: x[1] + 'L',ss)

'L_3 <- L_3 + -L_2'

In [257]:

def rand_diagonalisable(max_int=3,
             D=[1,2,3])->(np.array,np.array):
    
    '''returns 
    
    3x3 matrix M with
    - integer coefficients in [-max_int,max_int]
    - eigenvalues from the list D
    
    3x3 matrix A with
    - integer coefficients
    - M = A^{-1}DA
    
    '''
    
    A, ops = rand_inv()

    M = np.linalg.inv(A) @ np.diag(D) @ A
    
    #bit of a hack np.rint don't  work nice
    return np.round(M).astype(int), A
    

In [251]:
M, A = rand_diagonalisable([1,2,2])
M - M.astype(int)

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [255]:
np.linalg.det(M - 2*np.identity(3,dtype=int))

-3.108624468950436e-15

In [223]:
[round(x) for x in A.ravel()]

[-7, 2, 4, -23, 8, 8, -8, 2, 5]